#Pré-processamento dos dados da base de dados Sex_Offenders.csv

In [ ]:
# Importação das Bibliotecas Necessárias
import pandas as pd
from datetime import datetime, date
from geopy.geocoders import Nominatim
import numpy as np


In [ ]:
# Carregamento da Base de Dados de Agressores Sexuais
df = pd.read_csv("Sex_Offenders.csv")
df.head()

In [ ]:
# Remoção de Colunas de Nome e Sobrenome
# As colunas 'FIRST' e 'LAST' são removidas para proteger a privacidade dos indivíduos.
df.pop("FIRST")
df.pop("LAST")

In [ ]:
# Cálculo da Idade a partir da Data de Nascimento
def age(born):
    born = datetime.strptime(born, "%m/%d/%Y").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

df['AGE'] = df['BIRTH DATE'].apply(age)

In [ ]:
# Limpeza de Endereços Inconsistentes
df = df[df['BLOCK'].str.contains('HOMELESS') == False]
df = df[df['BLOCK'].str.contains('NaN') == False]
df = df[df['BLOCK'].str.contains('(registered as homeless)') == False]
df = df[df['BLOCK'].str.contains('(No value)') == False]

In [ ]:
geolocalizador = Nominatim(user_agent="Agressoes_sexuais")

In [ ]:
# Definição de Funções para Obtenção de Coordenadas
def pegarTodoEndereco(cordenada):
    localizacao = geolocalizador.reverse(cordenada, exactly_one=True)
    address = localizacao.raw['address']
    city = address.get('city', '')
    stage = address.get('stage', '')
    return city, stage

def pegarLatitudeeLongitude(block):
    global lat_long
    global block_remover
    try:
        novo_block = str(block)[4:]
        address = str(novo_block) + '. Chicago'
        location = geolocalizador.geocode(address)
        lat = location.latitude
        long = location.longitude
        city, stage = pegarTodoEndereco(f"{lat},{long}")
        if(city == 'Chicago'):
            lat_long.append([lat, long])
        else:
            lat_long.append([0, 0])
    except Exception as e:
        block_remover.append(block)


In [ ]:
# Aplicação das Funções para Obtenção das Coordenadas
lat_long = []
block_remover = []
df['BLOCK'].apply(pegarLatitudeeLongitude)

In [ ]:
# Remoção de Endereços Sem Coordenadas Válidas
for block in block_remover:
    df = df.drop(df[df['BLOCK'] == block].index)

In [ ]:
# Atribuição das Coordenadas ao DataFrame
lat_long = np.array(lat_long)
df['LAT'] = lat_long[:, 0]
df['LONG'] = lat_long[:, 1]

In [ ]:
# Limpeza Final do DataFrame
df = df.drop(df[df['LAT'] == 0].index)
df = df.reset_index(drop=True)